# Introduction

In this notebook, I do a few tests to analyze how racial demographics are correlated with income in New York City neighborhoods. I have placed census tract-level economic and social demographic data from the 2015 American Community Survey (ACS) into a MySQL table called "acs2015" in a database called "nyc." To analyze the data, I will be using PySpark.

Because I want to gain insights about how race and income are related, I will be mostly using simple models. Simpler models typically are less flexible and so may not give the strongest results, but they are often much easier to interpret.

Some things I use in this notebook:

  - PySpark
  - Spark SQL
  - Reading MySQL tables with Spark
  - SQL groupBy
  - SQL queries
  - Linear regression
  - K-Means clustering
  - k-Fold cross validation
  
As usual, we start by initializing the SparkContext and SQLContext.

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext('local[2]','Tutorial')
sql = SQLContext(sc)  


Now, I will use the JDBC (Java mysql connector) driver to read the database from the server.

In [173]:
acs2015 = sql.read.format('jdbc').options(
    url = 'jdbc:mysql://localhost/nyc',
    driver = "com.mysql.jdbc.Driver",
    dbtable = "acs2015",
    user="",
    password="").load()  

We now have the table in a DataFrame format.

printSchema() will show us the table schema.

In [174]:
acs2015.printSchema()

root
 |-- CensusTract: long (nullable = true)
 |-- County: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- TotalPop: long (nullable = true)
 |-- Men: long (nullable = true)
 |-- Women: long (nullable = true)
 |-- Hispanic: double (nullable = true)
 |-- White: double (nullable = true)
 |-- Black: double (nullable = true)
 |-- Native: double (nullable = true)
 |-- Asian: double (nullable = true)
 |-- Citizen: long (nullable = true)
 |-- Income: double (nullable = true)
 |-- IncomeErr: double (nullable = true)
 |-- IncomePerCap: double (nullable = true)
 |-- IncomePerCapErr: double (nullable = true)
 |-- Poverty: double (nullable = true)
 |-- ChildPoverty: double (nullable = true)
 |-- Professional: double (nullable = true)
 |-- Service: double (nullable = true)
 |-- Office: double (nullable = true)
 |-- Construction: double (nullable = true)
 |-- Production: double (nullable = true)
 |-- Drive: double (nullable = true)
 |-- Carpool: double (nullable = true)
 |-- Tran

I have added information such as income, total population, racial demographics, work and commute types, job categories, and more. We can look at a few rows.

In [175]:
acs2015.take(3)

[Row(CensusTract=36005000100, County='Bronx', Borough='Bronx', TotalPop=7703, Men=7133, Women=570, Hispanic=29.9, White=6.1, Black=60.9, Native=0.2, Asian=1.6, Citizen=6476, Income=None, IncomeErr=None, IncomePerCap=2440.0, IncomePerCapErr=373.0, Poverty=None, ChildPoverty=None, Professional=None, Service=None, Office=None, Construction=None, Production=None, Drive=None, Carpool=None, Transit=None, Walk=None, OtherTransp=None, WorkAtHome=None, MeanCommute=None, Employed=0, PrivateWork=None, PublicWork=None, SelfEmployed=None, FamilyWork=None, Unemployment=None),
 Row(CensusTract=36005000200, County='Bronx', Borough='Bronx', TotalPop=5403, Men=2659, Women=2744, Hispanic=75.8, White=2.3, Black=16.0, Native=0.0, Asian=4.2, Citizen=3639, Income=72034.0, IncomeErr=13991.0, IncomePerCap=22180.0, IncomePerCapErr=2206.0, Poverty=20.0, ChildPoverty=20.7, Professional=28.7, Service=17.1, Office=23.9, Construction=8.0, Production=22.3, Drive=44.8, Carpool=13.7, Transit=38.6, Walk=2.9, OtherTransp

Census tracts contain fairly consistent numbers of people, so we should see how many tracts we get in each of the five boroughs.

In [176]:
acs2015.groupBy('Borough').count().show()

+-------------+-----+
|      Borough|count|
+-------------+-----+
|       Queens|  669|
|     Brooklyn|  761|
|Staten Island|  110|
|    Manhattan|  288|
|        Bronx|  339|
+-------------+-----+



As expected, Brooklyn and Queens have the most. The Bronx actually has more tracts than Manhattan despite having a smaller population, but I would imagine that this is mostly due to the fact that Manhattan has a much higher population density.

We can use the pyspark.sql.functions package to look at aggregate functions such as the median household income of the wealthiest and poorest neighborhoods.

In [177]:
import pyspark.sql.functions as sqlf

acs2015.agg(sqlf.max('Income')).show()
acs2015.agg(sqlf.min('Income')).show()

+-----------+
|max(Income)|
+-----------+
|   244375.0|
+-----------+

+-----------+
|min(Income)|
+-----------+
|     9829.0|
+-----------+



# Running SQL Queries in PySpark

We can create a temp view of the DataFrame. This lets us run SQL queries on the DataFrame, which will make it much easier to create clean DataFrames. Here, I will look at things such as the total population and average income of tracts that have a majority of people in one of the four major racial categories, "White," "Black," "Hispanic," or "Asian."

In [178]:
acs2015.createOrReplaceTempView('ACS2015')

In [179]:
df = sql.sql('SELECT * from ACS2015 WHERE hispanic>50')
print('Count ' +str(df.count()))
df.groupBy('Borough').count().show()
df.agg(sqlf.sum('TotalPop')).show()
df.agg(sqlf.mean('Income')).show()
acs2015.agg(sqlf.mean('Income')).show()

Count 406
+-------------+-----+
|      Borough|count|
+-------------+-----+
|       Queens|   86|
|     Brooklyn|   65|
|Staten Island|    2|
|    Manhattan|   41|
|        Bronx|  212|
+-------------+-----+

+-------------+
|sum(TotalPop)|
+-------------+
|      1893465|
+-------------+

+-----------------+
|      avg(Income)|
+-----------------+
|36691.77306733167|
+-----------------+

+-----------------+
|      avg(Income)|
+-----------------+
|59101.32079961923|
+-----------------+



Majority Hispanic are most common in the Bronx and contain 1.9 million residents. These areas have an average income nearly 40% lower than the overall average income tract. Note that the averages here are not weighted by the number of residents, so this is not the same as the true average income for the city.

In [180]:
df = sql.sql('SELECT * from ACS2015 WHERE black>50')
print('Count ' +str(df.count()))
df.groupBy('Borough').count().show()
df.agg(sqlf.sum('TotalPop')).show()
df.agg(sqlf.mean('Income')).show()
acs2015.agg(sqlf.mean('Income')).show()

Count 454
+-------------+-----+
|      Borough|count|
+-------------+-----+
|       Queens|  118|
|     Brooklyn|  253|
|Staten Island|    3|
|    Manhattan|   27|
|        Bronx|   53|
+-------------+-----+

+-------------+
|sum(TotalPop)|
+-------------+
|      1617139|
+-------------+

+------------------+
|       avg(Income)|
+------------------+
|51464.282222222224|
+------------------+

+-----------------+
|      avg(Income)|
+-----------------+
|59101.32079961923|
+-----------------+



Majority black areas are most common in Brooklyn. They contain 1.6 million residents and have an average income around 13% lower than the city average.

In [181]:
df = sql.sql('SELECT * from ACS2015 WHERE white>50')
print('Count ' +str(df.count()))
df.groupBy('Borough').count().show()
df.agg(sqlf.sum('TotalPop')).show()
df.agg(sqlf.mean('Income')).show()
acs2015.agg(sqlf.mean('Income')).show()

Count 711
+-------------+-----+
|      Borough|count|
+-------------+-----+
|       Queens|  158|
|     Brooklyn|  288|
|Staten Island|   74|
|    Manhattan|  158|
|        Bronx|   33|
+-------------+-----+

+-------------+
|sum(TotalPop)|
+-------------+
|      2764109|
+-------------+

+-----------------+
|      avg(Income)|
+-----------------+
|80105.03299856528|
+-----------------+

+-----------------+
|      avg(Income)|
+-----------------+
|59101.32079961923|
+-----------------+



Majority white areas are common in every borough except the Bronx and contain 2.8 million residents. Their average income is about 1/3 higher than the city average.

In [182]:
df = sql.sql('SELECT * from ACS2015 WHERE asian>50')
print('Count ' +str(df.count()))
df.groupBy('Borough').count().show()
df.agg(sqlf.sum('TotalPop')).show()
df.agg(sqlf.mean('Income')).show()
acs2015.agg(sqlf.mean('Income')).show()

Count 100
+---------+-----+
|  Borough|count|
+---------+-----+
|   Queens|   70|
| Brooklyn|   22|
|Manhattan|    8|
+---------+-----+

+-------------+
|sum(TotalPop)|
+-------------+
|       429648|
+-------------+

+-----------+
|avg(Income)|
+-----------+
|   51281.32|
+-----------+

+-----------------+
|      avg(Income)|
+-----------------+
|59101.32079961923|
+-----------------+



Finally, majority Asian areas appear mainly in Queens and contain 400,000 residents. The average income there is very similar to the majority black areas, at around 13% below the average census tract.

So, if we consider areas with a majority of people from a single racial category, there are large differences in the average income depending on the racial category. These areas also contain roughly 3/4 of all NYC residents. This does make it appear that race has a strong correlation to income (although I haven't looked at things like the standard deviations). 

If we want to understand how a tract's racial demographics correlate with its income, linear regression would be a good first model.

# Income Prediction with Linear Regression

Linear regression is quite simple and generally easy to understand. If we don't add any polynomial features and just use the fraction of residents from each racial category, the coefficients tell us how much we expect the income to change as we alter the demographics of an area.

In [183]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

I will first get rid of null values. Then, I need to package the features as a Vector, which can be done with the VectorAssembler. I will only use three features: the percentage of residents identifying as black, Hispanic, and Asian. The sum of these three and the percentage of white residents is almost always very close to 100, so including all four would not really tell us any more information and might make it harder to find a good maximum. With this choice, the intercept tells us a "baseline" income for a nearly all-white neighborhood, and the coefficients tell us how each racial minority category changes the income.

In [184]:
acs2015 = acs2015.where(acs2015['Income'].isNotNull())

In [185]:
vectorizer = VectorAssembler(inputCols=['Black','Hispanic','Asian'],outputCol='features')

After adding the "features" column, we may as well drop the other columns that we're not going to need.

In [186]:
race_data = vectorizer.transform(acs2015)
race_data.printSchema()

root
 |-- CensusTract: long (nullable = true)
 |-- County: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- TotalPop: long (nullable = true)
 |-- Men: long (nullable = true)
 |-- Women: long (nullable = true)
 |-- Hispanic: double (nullable = true)
 |-- White: double (nullable = true)
 |-- Black: double (nullable = true)
 |-- Native: double (nullable = true)
 |-- Asian: double (nullable = true)
 |-- Citizen: long (nullable = true)
 |-- Income: double (nullable = true)
 |-- IncomeErr: double (nullable = true)
 |-- IncomePerCap: double (nullable = true)
 |-- IncomePerCapErr: double (nullable = true)
 |-- Poverty: double (nullable = true)
 |-- ChildPoverty: double (nullable = true)
 |-- Professional: double (nullable = true)
 |-- Service: double (nullable = true)
 |-- Office: double (nullable = true)
 |-- Construction: double (nullable = true)
 |-- Production: double (nullable = true)
 |-- Drive: double (nullable = true)
 |-- Carpool: double (nullable = true)
 |-- Tran

In [187]:
race_data = race_data.select('Income','features')
race_data.printSchema()

root
 |-- Income: double (nullable = true)
 |-- features: vector (nullable = true)



At this point, we're ready to run a linear regression model. I will optimize the regularization parameters, although it will turn out to not be very important.

Because I have not rescaled the features or the income, we will need a large regularization parameter to have much of an effect. In the Spark LinearRegression model, there are two parameters: An overall regularization strength parameter and another to give the balance between L1 and L2 regularization.

In [188]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

lr = LinearRegression(featuresCol='features',
                      labelCol='Income',
                      predictionCol='prediction',
                      maxIter=50,
                      regParam=0.2,
                      elasticNetParam=0.5)

(training, test) = race_data.randomSplit([0.8,0.2],seed=234)

paramGrid = ParamGridBuilder().addGrid(lr.regParam,[10,50,100,300,500,1000]) \
                              .addGrid(lr.elasticNetParam,[0.0,0.2,0.5,0.8,1]).build()
    
evaluator = RegressionEvaluator(metricName='r2',
                                labelCol='Income',
                                predictionCol='prediction')

cv = CrossValidator(estimator=lr,
                   estimatorParamMaps=paramGrid,
                   evaluator=evaluator,
                   numFolds=5)

mod = cv.fit(training)

In [189]:
vals = mod.avgMetrics
pars = mod.getEstimatorParamMaps()

for v,p in zip(vals,pars):
    r = list(p.values())[0]
    e = list(p.values())[1]
    
    print('Reg: {} ElNet: {} R2: {:0.4}'.format(r,e,v))

Reg: 10 ElNet: 0.0 R2: 0.38
Reg: 10 ElNet: 0.2 R2: 0.38
Reg: 10 ElNet: 0.5 R2: 0.38
Reg: 10 ElNet: 0.8 R2: 0.38
Reg: 10 ElNet: 1 R2: 0.38
Reg: 50 ElNet: 0.0 R2: 0.38
Reg: 50 ElNet: 0.2 R2: 0.38
Reg: 50 ElNet: 0.5 R2: 0.38
Reg: 50 ElNet: 0.8 R2: 0.38
Reg: 50 ElNet: 1 R2: 0.38
Reg: 100 ElNet: 0.0 R2: 0.38
Reg: 100 ElNet: 0.2 R2: 0.38
Reg: 100 ElNet: 0.5 R2: 0.3799
Reg: 100 ElNet: 0.8 R2: 0.3799
Reg: 100 ElNet: 1 R2: 0.3799
Reg: 300 ElNet: 0.0 R2: 0.3799
Reg: 300 ElNet: 0.2 R2: 0.3798
Reg: 300 ElNet: 0.5 R2: 0.3797
Reg: 300 ElNet: 0.8 R2: 0.3795
Reg: 300 ElNet: 1 R2: 0.3793
Reg: 500 ElNet: 0.0 R2: 0.3797
Reg: 500 ElNet: 0.2 R2: 0.3795
Reg: 500 ElNet: 0.5 R2: 0.3791
Reg: 500 ElNet: 0.8 R2: 0.3786
Reg: 500 ElNet: 1 R2: 0.3781
Reg: 1000 ElNet: 0.0 R2: 0.3789
Reg: 1000 ElNet: 0.2 R2: 0.3782
Reg: 1000 ElNet: 0.5 R2: 0.3766
Reg: 1000 ElNet: 0.8 R2: 0.3744
Reg: 1000 ElNet: 1 R2: 0.3725


We see here that less regularization is giving us better results. However, even at a regularization parameter of 1000, there is not much degradation in performance.

### Results

In [190]:
print('Coefficients: '+str(mod.bestModel.coefficients))
print('Intercept: '+str(mod.bestModel.intercept))

Coefficients: [-425.213390301,-770.430232518,-425.814647839]
Intercept: 95708.11321806749


Our fit shows an intercept of $95,708. This would be the expected income of an all-white neighborhood. It is also very high, perhaps skewed by extremely wealthy parts of Manhattan.

We see that the three linear coefficients are all negative. As the fraction of ethnic/racial minorities increases, the expected income falls. For both Asian and black populations, the expected income drops by $425 for every percent increase. Hispanic areas are the poorest, with the income dropping by $770 for every percent increase.

But, how has our model performed on the test set?

In [191]:
testpred = mod.transform(test)
r2 = evaluator.evaluate(testpred)
print('R2 on Test Set {:0.4}'.format(r2))

R2 on Test Set 0.333


We get an $R^2$ score of 0.33 here, somewhat smaller than the 0.38 that we expected from the cross validation. We didn't look at the individual cross validation numbers for the 5 folds, but it's fairly likely that this is well within the expected range.

An $R^2$ score of 0.33 shows that there likely is a correlation but it's not particularly strong. Racial demographics are only able to explain 1/3 of the variance in incomes between census tracts. Race is correlated with many other factors, so if we were to add more features, we might expect race to have a smaller effect.

We also only used basic linear features. The effect may be nonlinear, so adding more features even based just on our current set could give us a higher $R^2$ (in fact it must, which is why things like the adjusted $R^2$ exist). The disadvantage is that it becomes more difficult to interpret the numbers.

# Income Prediction with K-Means Clustering

Our linear regression model showed that we can find some correlation between race and income, but race alone can only explain maybe 1/3 of the variation. Another simple way that we could use to predict income is to use unsupervised learning techniques to split the data into clusters and then use the average income of each cluster as the predicted value.

The cluster properties may tell us about different sorts of neighborhoods that exist in the city. This cluster method is actually not so different from a decision tree regressor, but trained in an unsupervised manner.

In [192]:
from pyspark.ml.clustering import KMeans
import numpy as np

Once the data is in the right format, the Spark K-Means clustering class is pretty straightforward to use. I've just set K=10, but some optimization can obviously be done here.

In [193]:
km = KMeans().setK(10).setSeed(456)
model = km.fit(training)
err = np.sqrt(model.computeCost(training)/training.count())
transformed = model.transform(training)
transformed_test = model.transform(test)
print('Within Set RMSE: {:0.4}'.format(err))

Within Set RMSE: 11.54


Now that we've trained the data, we should look at some information about our clusters.

In K-Means, the clusters are defined by their center positions. We will also look at the count of tracts in each cluster to make sure that they look reasonably balanced, and finally, we will look at the mean income of tracts in each cluster.

In [194]:
centers = model.clusterCenters()

inc = transformed.groupby('prediction') \
           .agg(sqlf.avg('Income') \
           .alias('Income')) \
           .sort(sqlf.col('prediction')).collect()
num = transformed.groupby('prediction').count() \
                 .sort(sqlf.col('prediction')) \
                 .collect()
print('\tCluster Centers')
print('White/')
print('Other\tBlack\tHisp.\tAsian\tNum\tIncome')
for i,center in enumerate(centers):
    n = num[i]['count']
    income = inc[i].Income
    
    w = 100 - center[0]-center[1]-center[2]
    print('{:0.3}\t{:0.3}\t{:0.3}\t{:0.3}\t{}\t{:0.6}'
          .format(w,center[0],center[1],center[2],n,income))

	Cluster Centers
White/
Other	Black	Hisp.	Asian	Num	Income
8.99	31.7	55.6	3.68	134	31689.8
56.1	7.13	28.2	8.48	162	67940.6
6.36	82.4	9.37	1.88	253	56959.5
31.7	3.59	19.1	45.6	106	58261.2
81.2	2.48	9.07	7.28	328	85770.2
15.7	2.66	12.0	69.6	57	49392.1
56.0	5.09	12.8	26.1	193	72991.9
10.9	12.3	71.5	5.38	193	38460.5
16.9	52.6	24.9	5.59	184	43174.2
23.0	5.28	43.8	27.9	107	50418.8


We see that the smallest cluster has 57 tracts and the largest has 328, with most in the 100-200 range. We can immediately see a few things:

The smallest and two largest clusters have large majorities from a single racial group. The three majority-white clusters have by far the highest incomes.

The poorest cluster is nearly 90% black and Hispanic, while the next smallest is similar but also over 70% Hispanic. 

There are only two clusters where no racial category has a majority but no clusters where the typical tract has even just 10% or more residents from each of the four categories.

To make predictions, we need to take the predicted cluster and map it onto the income. Then, we can just run a RegressionEvaluator. The create_map() function is able to take a dictionary and then give us the mapped values.

In [195]:
from itertools import chain

theMap = {}
for idx,income in enumerate(inc):
    theMap[idx] = income.Income
mapper = sqlf.create_map( [sqlf.lit(p) for p in chain(*theMap.items())])
tfd = transformed_test.withColumn('PredInc',mapper.getItem(sqlf.col('prediction')))
tfd_train = transformed.withColumn('PredInc',mapper.getItem(sqlf.col('prediction')))

In [196]:
evaluator = RegressionEvaluator(metricName='r2',
                                labelCol='Income',
                                predictionCol='PredInc')
r2_train = evaluator.evaluate(tfd_train)
r2 = evaluator.evaluate(tfd)
print('Predicting Income from KMeans')
print('Training R2: {:0.4}'.format(r2_train))
print('Test R2: {:0.4}'.format(r2))

Predicting Income from KMeans
Training R2: 0.3675
Test R2: 0.3371


It turns out that this unsupervised model gives us essentially equivalent results as the linear regression classifier. It might be interesting to see what kinds of tracts do better with each regressor. If the results are different enough, perhaps something like a boosted decision tree model would be able to give us much better results.

# Conclusions

We've looked at how racial demographics in New York City appear to be related to income. From two different analyses, we've found that we can explain around 1/3 of the variation between different census tracts using racial demographics. So, while race does seem to be correlated with income, it is not the dominant factor explaining income levels. Our data set includes other useful information that we could add to the analysis or analyze separately, such as the kinds of jobs workers from each tract have.